In [1]:
import numpy as np
import scipy

def moment_legendre(n):
    return 1/(1+np.arange(n))

def jacobi_a_legendre(n):
    return 0.5*np.ones( (n,) )

def jacobi_b_legendre(n):
    k = 1.0*np.arange(1, n+1)
    b = 1/( 4*(4-k**(-2)) )
    #b = np.array( [1] + list( b ) )
    return np.sqrt(b)

def moment_laguerre(n):
    k = 1.0*np.arange(0, n)
    k[0] = 1.0
    return np.cumprod(k)

def jacobi_a_laguerre(n):
    k = 1.0*np.arange(1, n+1)
    return 2*k-1

def jacobi_b_laguerre(n):
    k = 1.0*np.arange(1, n+1)
    return k


In [7]:
compute_moments  = moment_legendre
compute_jacobi_a = jacobi_a_legendre
compute_jacobi_b = jacobi_b_legendre
scipy_quadrature = scipy.special.roots_legendre

# compute_moments  = moment_laguerre
# compute_jacobi_a = jacobi_a_laguerre
# compute_jacobi_b = jacobi_b_laguerre
# scipy_quadrature = scipy.special.roots_laguerre

## I. Direct approach: Cholesky

In [8]:
import freeDeconvolution

# Form moment matrix
moments_count = 5
mom_array = compute_moments( 2*moments_count + 1)

# Compute
jacobi_a, jacobi_b = freeDeconvolution.oprl.jacobi_from_moments( mom_array )

# Print Jacobi coefficients
print( "Computed Jacobi coefficients:")
print( "b: ", jacobi_b )
print( "a: ", jacobi_a )
print( "")

#jacobi_b = compute_jacobi_b(mom_count-1)
#jacobi_a = compute_jacobi_a(mom_count)
print( "Theoretical Jacobi coefficients:")
print( "b: ", compute_jacobi_b(moments_count-1) )
print( "a: ", compute_jacobi_a(moments_count) )
print( "")

Cholesky passed!

Diagonal of Cholesky
[1.         0.28867513 0.0745356  0.01889822 0.0047619  0.00119647]
Extra-diagonal of Cholesky
[0.5        0.28867513 0.1118034  0.03779645 0.01190476]

Computed Jacobi coefficients:
b:  [0.28867513 0.25819889 0.25354628 0.25197632]
a:  [0.5 0.5 0.5 0.5 0.5]

Theoretical Jacobi coefficients:
b:  [0.28867513 0.25819889 0.25354628 0.25197632]
a:  [0.5 0.5 0.5 0.5 0.5]



In [9]:
# Quadrature measure from our code
print("Quadrature measure")
support, weight = freeDeconvolution.quadrature_from_jacobi( jacobi_a, jacobi_b)
print( support )
print( weight )
# Compute weights via vandermonde (unstable)
# vandermonde = np.vander( eigen ).T
# weights = np.linalg.solve( vandermonde, mom_array[0:mom_count] )
# print( "Raw Weights: ")
# print( weights)
print("")

# Quadrature from scipy
print( "Ground truth")
import scipy
points, weights, mass = scipy_quadrature( len(support), mu=True)
if scipy_quadrature == scipy.special.roots_legendre:
    points = (points+1)/2
print( points )
weights = weights/weights.sum()
print( weights )

Quadrature measure
[0.95308992 0.76923466 0.5        0.04691008 0.23076534]
[0.11846344 0.23931434 0.28444444 0.11846344 0.23931434]

Ground truth
[0.04691008 0.23076534 0.5        0.76923466 0.95308992]
[0.11846344 0.23931434 0.28444444 0.23931434 0.11846344]
